## Phase 3 (Data Preparation): SibSp und Parch => FamilySizeBinned

* Autorin: Anna (i3-Versicherung)
* Webseite: [Data Science Training - Kapitel 5](https://data-science.training/kapitel-5/)
* Datum: 23.03.2023

Aus den beiden Attributen "SibSp" (Geschwister und Ehepartner) und "Parch" (Eltern und Kinder) können wir "FamilySize" (Familiengröße) als neues Feature berechnen. Mit Hilfe von Binning können wir anschließend aus diesem kardinalskalierten Feature ein ordinalskaliertes machen.

In [4]:
# Pandas Paket (Package) importieren
#  Datenstrukturen und Datenanalyse, I/O
#  https://pandas.pydata.org/pandas-docs/stable/
import pandas as pd

In [5]:
# Trainings- und Testdaten als Pandas Data Frame (df) aus CSV-Dateien laden
#  (KNIME: "CSV Reader")
df_train = pd.read_csv('../../data/titanic/original/train.csv')
df_test  = pd.read_csv('../../data/titanic/original/test.csv')

In [6]:
# Trainings- und Testdaten zusammenführen
#  (KNIME "Concatenate")
#df = pd.concat([df_train, df_test], ignore_index=True)
# Wir benutzen nur die Trainingsdaten
df = df_train

In [7]:
# Datentypen automatisch konvertieren
df = df.convert_dtypes()

In [8]:
# Spezielle Titanic-Statistik (Überlebenswahrscheinlichkeiten)
def dst_titanic_statistics(df, col):
    stats = df.groupby([col], as_index=False, observed=True)['PassengerId'].count() # Spalte "PassengerId" aus Titanic-Daten
    probs = df.groupby([col], as_index=False, observed=True)['Survived'].mean()     # Spalte "Survived"    aus Titanic-Daten
    stats = stats.merge(probs, on=col)
    stats = stats.rename(columns={'PassengerId': 'Anzahl', 'Survived': 'Überlebenswahrscheinlichkeit'})
    return stats

In [9]:
# Neues Feature "FamilySize"
#  (KNIME: "Math Formula")
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
# Statistik (Überlebenswahrscheinlichkeiten)
stats = dst_titanic_statistics(df, 'FamilySize')
display(stats)

,FamilySize,Anzahl,Überlebenswahrscheinlichkeit
0,1,537,0.303538
1,2,161,0.552795
2,3,102,0.578431
3,4,29,0.724138
4,5,15,0.2
5,6,22,0.136364
6,7,12,0.333333
7,8,6,0.0
8,11,7,0.0


#### Schlussfolgerungen und Idee

Die Überlebenswahrscheinlichkeit ist bei einer Familiengröße von 4 Personen am höchsten (72 %).

Wir können ein manuelles Binning vornehmen und Familiengrößen mit ähnlichen Überlebenswahrscheinlichkeiten zu einem Bin zusammenfassen:

| FamilySize | P(Survived=1)       |
| -----------| --------------------|
|1           | 0,304               |
|2, 3        | 0,553; 0,578        |
|4           | 0,724               |
|5, 6, 7     | 0,200; 0,136; 0,333 |
|8 - 99      | 0,000               |

In [11]:
# FamilySize Binning: Manuelles Binning mit 5 Bins (v1)
bins = [1, 2, 4, 5, 8, 99]
labels = ['No', 'Small', 'Medium','Large', 'Huge']
df['FamilySizeBinned'] = pd.cut(df['FamilySize'], bins, right=False, labels=labels)
# Statistik (Überlebenswahrscheinlichkeiten)
stats = dst_titanic_statistics(df, 'FamilySizeBinned')
display(stats)

,FamilySizeBinned,Anzahl,Überlebenswahrscheinlichkeit
0,No,537,0.303538
1,Small,263,0.562738
2,Medium,29,0.724138
3,Large,49,0.204082
4,Huge,13,0.0


#### Schlussfolgerungen

Die 5 Bins sind sehr ungleich verteilt. Fast alle Passagiere befinden sich in den ersten beiden Bins.

Gegebenenfalls können wir hier noch weiter Vereinfachen und Small und Medium sowie Lage und Huge zusammenfassen.

In [13]:
# FamilySize Binning: Manuelles Binning mit nur 3 Bins (v2)
bins = [1, 2, 5, 99]
labels = ['No', 'Small', 'Large']
df['FamilySizeBinned'] = pd.cut(df['FamilySize'], bins, right=False, labels=labels)
# Statistik (Überlebenswahrscheinlichkeiten)
stats = dst_titanic_statistics(df, 'FamilySizeBinned')
display(stats)

,FamilySizeBinned,Anzahl,Überlebenswahrscheinlichkeit
0,No,537,0.303538
1,Small,292,0.578767
2,Large,62,0.16129


In [14]:
display(df.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,FamilySizeBinned
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,<NA>,S,2,Small
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,Small
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,<NA>,S,1,No
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S,2,Small
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,<NA>,S,1,No


### Ergebnis

Kleine Familien haben die höchste Überlebenswahrscheinlichkeit. Die meisten Passagiere der Titanic sind jedoch allein gereist. Die neuen Features FamilySize und FamilySizeBinned (v2) sollten geeignet sein, um diesen Aspekt zu berücksichtigen.